In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://www.nikkei.com/")

try:
    # WebDriverWaitを使って最大10秒間待機
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.body_bgvwx9q > div.container_clyllbs > main > div:nth-child(1) > div > div:nth-child(1) > div:nth-child(1) > div > div > article > div.textContainer_t1v5liye.overlayStyleOfTextContainer_o1ezlocp.leftAlignedTextContainer_lgmry3u > h2"))
    )
    print(element.text)
finally:
    driver.quit()

中国恒大の法的整理、判断を24年1月に延期香港高裁


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ChromeのWebドライバを起動。システムパスから動かしている
driver = webdriver.Chrome()

# スクレイピング対象のURLを開く
driver.get('https://www.nikkei.com/')

# 特定の要素を取得
# 　　・driver.find_element(By.CSS_SELECTOR, ".foo")

element = driver.find_element(By.CSS_SELECTOR, 'body > div.body_bgvwx9q > div.container_clyllbs > main > div:nth-child(1) > div > div:nth-child(1) > div:nth-child(1) > div > div > article > div.textContainer_t1v5liye.overlayStyleOfTextContainer_o1ezlocp.leftAlignedTextContainer_lgmry3u > h2')
print(element.text)

# ブラウザを閉じる
driver.quit()

中国恒大の法的整理、判断を24年1月に延期香港高裁


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 処理するURLのリスト
urls = [
    "https://www.scmp.com/infographic/",
    "https://example.com/page1",
    "https://example.com/page2",
    # 他のURLをここに追加
]

# Chromeドライバのインスタンスを作成
driver = webdriver.Chrome()

# 各URLに対して処理を実行
for url in urls:
    try:
        driver.get(url)

        # WebDriverWaitを使って最大10秒間待機
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#latest0 > a > h1"))
        )
        print(f"URL: {url} - Element Text: {element.text}")
    except Exception as e:
        print(f"Error occurred while processing URL: {url}")
        print(e)

# 全ての処理が終わった後、ドライバを終了
driver.quit()

URL: https://www.scmp.com/infographic/ - Element Text: Are gates to hell open? What 2 degrees warmer means for Earth
Error occurred while processing URL: https://example.com/page1
Message: 
Stacktrace:
0   chromedriver                        0x000000010d6ebd28 chromedriver + 4795688
1   chromedriver                        0x000000010d6e32b3 chromedriver + 4760243
2   chromedriver                        0x000000010d2bc88d chromedriver + 407693
3   chromedriver                        0x000000010d30b3d9 chromedriver + 730073
4   chromedriver                        0x000000010d30b591 chromedriver + 730513
5   chromedriver                        0x000000010d3517e4 chromedriver + 1017828
6   chromedriver                        0x000000010d332ccd chromedriver + 892109
7   chromedriver                        0x000000010d34eb2b chromedriver + 1006379
8   chromedriver                        0x000000010d332a73 chromedriver + 891507
9   chromedriver                        0x000000010d2fd143 chrome